In [58]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.parse
import json
import requests
import re
import os

batch_size = 200

filename = 'DataDictFull.json'

first_scan = True

if os.path.exists(filename):
    file_handle = open(filename)
    date_detail = json.loads(file_handle.read())
    print("Total Size: ", len(date_detail.keys()))

    counter = 0
    empty = 0
    finished = 0
    for dict_key in date_detail.keys():
        row = date_detail[dict_key]
        if 'Composition Date' in row:
            finished += 1
            if row['Composition Date'] == '':
                if not first_scan and 'url' in row.keys():
                    url = row['url']
                    r = requests.get(url).content
                    soup = BeautifulSoup(r, 'html.parser')
                    target_th = soup.find(string=re.compile('Info.'))
                    if target_th:
                        year_string = target_th.find_next('td').text.strip()
                        year = re.search(r"(1[4-9]\d{2}|20\d{2})", year_string)
                        composition_date = ""
                        if year:
                            composition_date = year.group()
                        
                        if composition_date:
                            print(composition_date)
                            date_detail[dict_key]['Composition Date'] = composition_date
                            counter += 1
                            print(counter)
                            if counter >= batch_size:
                                break
                        else:
                            empty += 1
                    else:
                        empty += 1
                if first_scan:
                    empty += 1
            continue
        
        music = row['Piece_Name']
        music = re.sub(r'\s+', '_', music)
        composer = "_(" + row['Surname'] + ",_" + row['First_Name'] + ")"
        url = "https://imslp.org/wiki/" + urllib.parse.quote(music + composer)
        date_detail[dict_key]['url'] = url
        print(url)

        composition_date = ""

        r = requests.get(url).content
        soup = BeautifulSoup(r, 'html.parser')
        target_th = soup.find(string=re.compile('Composition Year'))
        if not target_th:
            target_th = soup.find(string=re.compile('First Publication'))
        if target_th:
            year_string = target_th.find_next('td').text.strip()
            year = re.search(r"(1[4-9]\d{2}|20\d{2})", year_string)
            if year:
                composition_date = year.group()
        if composition_date == '':
            target_th = soup.find(string=re.compile('Info.'))
            if target_th:
                year_string = target_th.find_next('td').text.strip()
                year = re.search(r"(1[4-9]\d{2}|20\d{2})", year_string)
                if year:
                    composition_date = year.group()
        if composition_date == '':
            empty += 1
        date_detail[dict_key]['Composition Date'] = composition_date
        counter += 1
        finished += 1
        if counter >= batch_size:
            break
    print("Total Finished: ", finished)
    print("Total Empty: ", empty)
    file_handle.close()

    with open(filename, 'w') as convert_file:
        convert_file.write(json.dumps(date_detail))


Total Size:  10854
Total Finished:  10854
Total Empty:  1183


In [59]:
filename = 'DataDictFull.json'

if os.path.exists(filename):
    file_handle = open(filename)
    date_detail = json.loads(file_handle.read())

    invalid_url = 0
    for dict_key in date_detail.keys():
        row = date_detail[dict_key]
        if 'Composition Date' in row and row['Composition Date'] == '':
            url = row['url']
            r = requests.get(url).content
            soup = BeautifulSoup(r, 'html.parser')
            target_cell = soup.find(string=re.compile('currently no text in this page'))
            if target_cell:
                invalid_url += 1
                print(url)

    print("Total Invalid Url: ", invalid_url)
    file_handle.close()

https://imslp.org/wiki/Biblioteca_d%27oro_%28Longo%2C_Alessandro%29_%28Capponi%2C_Francesco%29
https://imslp.org/wiki/Me_Amas_%2C_Op.111_%28Alba%2C_Antonio%29
https://imslp.org/wiki/5_Studies%2C_Anh.1a_1_%28Brahms%2C_Johannes%29
https://imslp.org/wiki/Rondino_in_A-flat_major_%28Andersen%2C_Harris%29
https://imslp.org/wiki/Transcription_-_Bach__Toccata_and_Fugue_in_D_Minor_%28BWV_565%29_%28Brassin%2C_Louis%29
https://imslp.org/wiki/Study_for_the_Left_Hand%2C_Anh.4_2_%28Brahms%2C_Johannes%29
https://imslp.org/wiki/32_3%2C_Op.32_%28Aramaki%2C_Michio%29
https://imslp.org/wiki/Allemande_in_G_minor%2C_BWV_836_%28Bach%2C_Johann_Sebastian%29_%28Bach%2C_Wilhelm%20Friedemann%29
https://imslp.org/wiki/Im_3_4_Takt%2C_Op.48_%28Bortkiewicz%2C_Sergei%29
https://imslp.org/wiki/6_Romances_sans_paroles_%28Mendelssohn%2C_Felix%29_%28Belloc%2C_E%CC%81mile%29
https://imslp.org/wiki/Why_do_Summer_Roses_Fade__%28Barker%2C_George%20Arthur%29
https://imslp.org/wiki/Fantasia_su_%E2%80%98Akai_kutsu%E2%80%99_%28C